In [ ]:
import numpy as np
import pandas as pd

filename = "/Users/joshkim/Desktop/math 104/dataset/ydata-ymusic-user-artist-ratings-v1_0.txt"
filename2 = "/Users/joshkim/Desktop/math 104/dataset/ydata-ymusic-artist-names-v1_0.txt"

In [ ]:
# This reads the file with artist ratings, artist id, and user id
df = pd.read_csv(filename, sep = '\s+', header = None)

In [ ]:
# This reads the file with artist id and artist names
df2 = pd.read_csv(filename2, sep = '\t', header = None, encoding='latin-1')
print(df2.head())
print(df2.shape[0])

In [ ]:
print(df.head())
print(df.shape[0])
print(df.shape[0] * df.shape[1])

In [ ]:
# Look for users who have reviewed more than 50 artists
# usercount = df[[1, 0]].groupby(0).count()
# usercount = usercount[usercount[1] >= 50]
# print(usercount.head())


In [ ]:
# Look for artists reviewed by more than 50 users

# artistcount = df[[1,0]].groupby(1).count()
# artistcount = artistcount[artistcount[0] >= 50]
# print(artistcount.head())

In [ ]:
# Keep only the popular artists and users
# df = df[df[0].isin(usercount.index)] & df[1].isin(artistcount.index)]
print(df)
print(df.shape[0])
print(df.shape[0] * df.shape[1])
# Keep only a segment of the data (can't do the whole thing jupyter won't allow)
df = df[0:100000]

In [ ]:
# This creates a matrix with the users representing the columns and the artists representing the rows with 
df = df.pivot(index = 1, columns = 0, values = 2)
# This fills all of the NaN slots with 0s
df = df.fillna(0)
print(df)

In [ ]:
# This calculates the RSVD of the Ratings Matrix
U, S, V = np.linalg.svd(df, full_matrices = False)

In [ ]:
# This constructs a low rank approximation Matrix with the new Sigma being the first k singular values
k=10
Sigma = np.diag(S[:k])
# Computes the low rank approximation of A using the first k singular values
A_lowrank = U[:, :k] @ Sigma @ V[:k, :]
U = U[:, :k]
V = V[:k, :]

In [ ]:
# Get the number of rows in A_lowrank
m = A_lowrank.shape[0]
print(m)

In [ ]:
# Create a vector of zeros with length equal to the number of rows in A


In [ ]:
# Vector of artists that I like - sourced from Apple Music Wrapped
v = ["Kanye West", "David Bowie", "Red Hot Chili Peppers", "Train", "The Beatles", "Led Zeppelin", "Jay-Z", "Maroon 5", "Jason Mraz", "Queen"]
indices = []

# Searches through the artist list and finds indices
for i in range(len(v)):
    for j in range(2,97956):
        if df2.iloc[j][1]== v[i]:
            print (j, df2.iloc[j][1])
            indices.append(j)
print(indices)

# Column with all of the 
Artists = list(df.index)
print(Artists)
for i in range(len(Artists)):
    if(Artists[i] not in indices):
        Artists[i] = 0
    if (Artists[i] in indices):
        Artists[i] = 100
print(Artists)

    
    

In [ ]:
# This puts the new vector in terms of Rk
# Compute the inverse of S

S_inv = np.linalg.inv(Sigma)
u = np.array(Artists)
# Compute the matrix product u' = u^T * U * S^-1
u_prime = u.T @ U @ S_inv

In [ ]:
# Check the similarities
def cosine_similarity(v,u):
    return (v @ u)/ (np.linalg.norm(v) * np.linalg.norm(u))
 
highest_similarity = -np.inf
highest_sim_col = -1
for col in range(1,V.shape[1]):
    similarity = cosine_similarity(V[:,0], V[:,col])
    if similarity > highest_similarity:
        highest_similarity = similarity
        highest_sim_col = col
 
print("Column %d (Artist id %s) is most similar to column 0 (Artist id %s)" %
        (highest_sim_col, A_lowrank.columns[col], A_lowrank.columns[0])
)